In [1]:
# Created by: Sergiu Iatco
# Created on: 16.05.2018

In [2]:
# Source:
# AI in Marketing - Siraj Raval
# https://www.youtube.com/watch?v=FYMjXD3G__Y
# https://github.com/llSourcell/AI_In_Marketing 

In [3]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

'''
AI for script generating in Romanian based on novel of Ion Creanga
Code is adapted to convert diacritics to latin (unidecode)
Script is generated at the given step (print_step)
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

#path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
#with io.open(path, encoding='utf-8') as f:
#    text = f.read().lower()

file = 'Amintiri_din_copilarie.txt'
tx = open(file, encoding='latin-1', mode = 'r') 
txr = tx.read().lower()
accented_string = txr
# accented_string is of type 'unicode'

import unidecode
unaccented_string = unidecode.unidecode(accented_string)
text = unaccented_string

print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
print_step = 20 # At which step to generate the text

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    if (epoch +1) %  print_step == 0:
        # Function invoked at end of each epoch. Prints generated text.
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=300,
          callbacks=[print_callback])

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


corpus length: 140716
total chars: 45
nb sequences: 46892
Vectorization...
Build model...
Epoch 1/300
46892/46892 [==============================] - 26s 545us/step - loss: 2.3636
Epoch 2/300
46892/46892 [==============================] - 26s 547us/step - loss: 2.0370
Epoch 3/300
46892/46892 [==============================] - 27s 566us/step - loss: 1.8932
Epoch 4/300
46892/46892 [==============================] - 26s 549us/step - loss: 1.7896
Epoch 5/300
46892/46892 [==============================] - 26s 545us/step - loss: 1.6941
Epoch 6/300
46892/46892 [==============================] - 26s 548us/step - loss: 1.6236
Epoch 7/300
46892/46892 [==============================] - 26s 550us/step - loss: 1.5697
Epoch 8/300
46892/46892 [==============================] - 26s 561us/step - loss: 1.5047
Epoch 9/300
46892/46892 [==============================] - 26s 550us/step - loss: 1.4670
Epoch 10/300
46892/46892 [==============================] - 26s 549us/step - loss: 1.4154
Epoch 11/300
46892/

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in log


-a luatatza, si
bagirig pe la intoarcem, cu este mos bodragii, ca de mama lupr vede era si odana cu zice tot cinev surtul vradimepe daca
in alta schimine, laperimete ce doatevatatul laporle sand-lomul
----- diversity: 1.2
----- Generating with seed: "cus va ard cateva jordii prin tolul cela"
cus va ard cateva jordii prin tolul cela pe
corea perele dreptesorul trama, pariste, poatejoaste erau si ma dum invetini, intre azi la gramatine smintelete lanva desage, caci tata, suita lacuri de crede mai mi-ad chiga te voatema asa alta oslovala.
dar elatescora, zicea domnura-ma
noida mine: ca dimine luhr-me-ma dau iefe strumnogeaca si
drapuluit pei, fehere altei potid de bubta ce aveai: canda-n gura,
vaienii. s-apoi cand m-am iota eu
Epoch 21/300
46892/46892 [==============================] - 26s 554us/step - loss: 1.2460
Epoch 22/300
46892/46892 [==============================] - 26s 554us/step - loss: 1.2398
Epoch 23/300
46892/46892 [==============================] - 26s 545us/step - loss: 1.2

elui, cum era sprintara si plina de incurmare, levura peste tihinataturi cgela piem stricinta, dupa unelui, garniri adata, hai mazece toaceam cateau bumina, asumna! bagand de targhil-born din bos luca iacese... casema
cura: las' tabim ne gaste ciuca si mi-o
ori
cam prin prin capus in viatandu-le... fieca-ne multe altii, tamal o
il, din flazul laga ti-o cuva!
mai omani, popoli la
scola iosa nu fugitica ce am facut popul ape supa undeva..
Epoch 61/300
46892/46892 [==============================] - 26s 548us/step - loss: 0.9644
Epoch 62/300
46892/46892 [==============================] - 26s 556us/step - loss: 0.9647
Epoch 63/300
46892/46892 [==============================] - 26s 552us/step - loss: 0.9500
Epoch 64/300
46892/46892 [==============================] - 26s 546us/step - loss: 0.9556
Epoch 65/300
46892/46892 [==============================] - 26s 546us/step - loss: 0.9473
Epoch 66/300
46892/46892 [==============================] - 26s 547us/step - loss: 0.9396
Epoch 67/300
46892/

cate unul, in toata ziule gatlan, si fuga acasa, cap ca o cusai de fatlavlaros, imi zicea am atunci era agasca! oriva smantarula frumoasa aflai iins dregul prin humulea: nu put treaca atata mase cu o veilin parintelum, mogros si bochiorund in colo si o cheurojul maluie sa-i fac pentru-cu din
greua, harbunda ca nu puta, mie mai artogopi
Epoch 141/300
46892/46892 [==============================] - 25s 543us/step - loss: 0.7130
Epoch 142/300
46892/46892 [==============================] - 26s 545us/step - loss: 0.7171
Epoch 143/300
46892/46892 [==============================] - 26s 553us/step - loss: 0.7089
Epoch 144/300
46892/46892 [==============================] - 26s 545us/step - loss: 0.7032
Epoch 145/300
46892/46892 [==============================] - 26s 544us/step - loss: 0.7026
Epoch 146/300
46892/46892 [==============================] - 26s 544us/step - loss: 0.7068
Epoch 147/300
46892/46892 [==============================] - 26s 548us/step - loss: 0.7032
Epoch 148/300
46892/46892

si eu, rau! unii cas decat in pricina oaseni prin carele
sa ia prind: mai tihep la toate zeunda, de-l noi
Epoch 221/300
46892/46892 [==============================] - 26s 558us/step - loss: 0.6052
Epoch 222/300
46892/46892 [==============================] - 26s 544us/step - loss: 0.6052
Epoch 223/300
46892/46892 [==============================] - 26s 547us/step - loss: 0.6068
Epoch 224/300
46892/46892 [==============================] - 26s 544us/step - loss: 0.6058
Epoch 225/300
46892/46892 [==============================] - 26s 550us/step - loss: 0.6070
Epoch 226/300
46892/46892 [==============================] - 26s 552us/step - loss: 0.6005
Epoch 227/300
46892/46892 [==============================] - 26s 544us/step - loss: 0.6051
Epoch 228/300
46892/46892 [==============================] - 26s 545us/step - loss: 0.6028
Epoch 229/300
46892/46892 [==============================] - 26s 545us/step - loss: 0.6042
Epoch 230/300
46892/46892 [==============================] - 26s 558us/step

In [4]:
# The generated text is not coherent
# Question: What is necessary to make the code work for Romanian language or other latin language?